In [3]:
import numpy as np
import librosa

# Feature extraction

In [7]:
def extract_features(data, sr):
    # Zero Crossing Rate
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    
    # Chroma (STFT)
    stft = np.abs(librosa.stft(data))
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
    
    # Mel Frequency Cepstral Coefficients
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sr, n_mfcc=40).T, axis=0)
    
    # Stack them into one long array
    # Shape: (1 + 12 + 40) = 53 features
    result = np.hstack((zcr, chroma, mfccs))
    return result

# Preparing the data for CNN